# 簡單的RNN實作

## 程式參考來源：
- https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html
- https://pytorch.org/docs/stable/generated/torch.nn.RNN.html#torch.nn.RNN
- https://pytorch.org/text/stable/vocab.html
- https://pytorch.org/text/stable/functional.html#to-tensor
- https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html


## 載入相關套件

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import numpy as np

## 嵌入層測試

In [3]:
x = torch.LongTensor([[0,1,2], [3,4,5]])
embeds = nn.Embedding(6, 5) 
print(embeds(x))

tensor([[[-0.2854,  0.4994, -1.2292,  0.0285, -1.4484],
         [-0.4937,  0.7987, -0.5471,  1.5526,  1.3826],
         [-0.1778, -1.9945,  0.3916,  0.7550,  0.2322]],

        [[ 0.2465,  0.7877,  0.3312,  0.5031, -0.3601],
         [ 0.9708,  0.3138, -0.4496,  1.8550,  0.6466],
         [ 2.1568,  0.5826, -1.4558,  0.1674,  1.6133]]],
       grad_fn=<EmbeddingBackward0>)


In [4]:
embeds.weight

Parameter containing:
tensor([[-0.2854,  0.4994, -1.2292,  0.0285, -1.4484],
        [-0.4937,  0.7987, -0.5471,  1.5526,  1.3826],
        [-0.1778, -1.9945,  0.3916,  0.7550,  0.2322],
        [ 0.2465,  0.7877,  0.3312,  0.5031, -0.3601],
        [ 0.9708,  0.3138, -0.4496,  1.8550,  0.6466],
        [ 2.1568,  0.5826, -1.4558,  0.1674,  1.6133]], requires_grad=True)

In [16]:
x = torch.LongTensor([[1,2,3], [4,5,6]])
embeds = nn.Embedding(7, 5) 
print(embeds(x))

tensor([[[-0.2021, -1.0623,  0.0677,  0.8205,  0.7972],
         [ 1.8586, -0.6237,  0.9995, -0.7118,  0.2388],
         [-1.8304, -0.7225, -0.9457, -0.3937,  0.3253]],

        [[ 0.9283,  1.0477,  0.8933, -0.2366, -0.7531],
         [-1.5280,  0.4154,  0.2127,  1.0361,  0.9087],
         [-1.5227,  0.3780,  0.1096,  1.3931, -1.5150]]],
       grad_fn=<EmbeddingBackward0>)


In [53]:
embeds = nn.Embedding(6, 5) 
x1 = torch.LongTensor([[0,1,2]])
x2 = torch.LongTensor([[3,4]])
print(embeds(x1))
print(embeds(x2))
embeds.weight

tensor([[[ 2.6216, -2.1906, -0.0539,  0.1866,  0.1487],
         [-1.3029,  0.7337, -0.1752,  0.2191,  0.7611],
         [-1.0816,  0.1376,  2.1151,  1.3313,  0.3332]]],
       grad_fn=<EmbeddingBackward0>)
tensor([[[ 1.6070,  0.5820,  0.4085, -1.8771,  0.0628],
         [-0.5556, -1.2174, -0.7017, -0.1620,  0.8118]]],
       grad_fn=<EmbeddingBackward0>)


Parameter containing:
tensor([[ 2.6216, -2.1906, -0.0539,  0.1866,  0.1487],
        [-1.3029,  0.7337, -0.1752,  0.2191,  0.7611],
        [-1.0816,  0.1376,  2.1151,  1.3313,  0.3332],
        [ 1.6070,  0.5820,  0.4085, -1.8771,  0.0628],
        [-0.5556, -1.2174, -0.7017, -0.1620,  0.8118],
        [ 0.0888,  0.6752,  1.1217, -0.4797, -0.1823]], requires_grad=True)

In [29]:
embeds = nn.Embedding(6, 5, 5) 
x1 = torch.LongTensor([[0,1,2]])
x2 = torch.LongTensor([[3,4]])
x3 = torch.LongTensor([[3,4]])
print(embeds(x1))
print(embeds(x2))
print(embeds(x3))
embeds.weight

tensor([[[-1.1070, -0.0912, -1.6728, -0.8792,  0.5309],
         [-0.9439,  1.1110,  0.0358, -0.9717,  2.7881],
         [-0.3279, -0.7180, -1.7330, -1.2374,  1.7251]]],
       grad_fn=<EmbeddingBackward0>)
tensor([[[-0.0248,  0.6468, -0.8845, -1.1123, -0.3905],
         [-0.1012,  0.2742, -0.2391,  1.9627,  1.2456]]],
       grad_fn=<EmbeddingBackward0>)
tensor([[[-0.0248,  0.6468, -0.8845, -1.1123, -0.3905],
         [-0.1012,  0.2742, -0.2391,  1.9627,  1.2456]]],
       grad_fn=<EmbeddingBackward0>)


Parameter containing:
tensor([[-1.1070, -0.0912, -1.6728, -0.8792,  0.5309],
        [-0.9439,  1.1110,  0.0358, -0.9717,  2.7881],
        [-0.3279, -0.7180, -1.7330, -1.2374,  1.7251],
        [-0.0248,  0.6468, -0.8845, -1.1123, -0.3905],
        [-0.1012,  0.2742, -0.2391,  1.9627,  1.2456],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]], requires_grad=True)

In [2]:
# 測試資料
word_to_ix = {"hello": 0, "world": 1}
# 詞彙表(vocabulary)含2個單字, 轉換為5維的向量
embeds = nn.Embedding(2, 5) 
# 測試 hello
lookup_tensor = torch.LongTensor([word_to_ix["hello"]])
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[-0.4981,  1.1051,  1.0376,  0.2448,  0.6538]],
       grad_fn=<EmbeddingBackward0>)


## RNN層測試

In [348]:
torch.randn(5, 3, 10).shape

torch.Size([5, 3, 10])

In [349]:
# 測試資料
input = torch.randn(5, 10)
# 建立 RNN 物件
rnn = nn.RNN(10, 20, 2)
# RNN 處理
output, hn = rnn(input)
# 顯示輸出及隱藏層的維度
print(output.shape, hn.shape)

torch.Size([5, 20]) torch.Size([2, 20])


In [350]:
# 測試資料
input = torch.randn(5, 4, 10)
# 建立 RNN 物件
rnn = nn.RNN(10, 20, 2)
# RNN 處理
output, hn = rnn(input)
# 顯示輸出及隱藏層的維度
print(output.shape, hn.shape)

torch.Size([5, 4, 20]) torch.Size([2, 4, 20])


In [351]:
# 測試資料
input = torch.randn(5, 3, 10)
# 建立 RNN 物件
rnn = nn.RNN(10, 20, 2)
# 隱藏層的輸入
h0 = torch.randn(2, 3, 20)
# RNN 處理
output, hn = rnn(input, h0)
# 顯示輸出及隱藏層的維度
print(output.shape, hn.shape)

torch.Size([5, 3, 20]) torch.Size([2, 3, 20])


## 分詞

In [352]:
from torchtext.data.utils import get_tokenizer

tokenizer = get_tokenizer('basic_english')

text = 'Could have done better.'        
tokenizer(text)

['could', 'have', 'done', 'better', '.']

## 詞彙表處理

In [353]:
from torchtext.vocab import vocab
from collections import Counter, OrderedDict

# BOW 統計
counter = Counter(tokenizer(text))
# 依出現次數降冪排列
sorted_by_freq_tuples = sorted(counter.items(), 
                       key=lambda x: x[1], reverse=True)
# 建立詞彙字典
ordered_dict = OrderedDict(sorted_by_freq_tuples)

# 建立詞彙表物件，並加一個未知單字(unknown)的索引值
vocab_object = torchtext.vocab.vocab(ordered_dict, specials=["<unk>"])
# 設定詞彙表預設值為未知單字(unknown)的索引值
vocab_object.set_default_index(vocab_object["<unk>"])

# 測試
vocab_object['done']

3

In [354]:
vocab_object.get_itos()

['<unk>', 'could', 'have', 'done', 'better', '.']

In [355]:
vocab_object.__len__()

6

In [356]:
import string

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [357]:
import string

def create_vocabulary(text_list):
    # 取得標點符號
    stopwords = list(string.punctuation)
    
    # 去除標點符號
    clean_text_list = []
    clean_tokens_list = []
    for text in text_list:
        tokens = tokenizer(text) 
        clean_tokens = []
        for w in tokens:
            if w not in stopwords:
                clean_tokens.append(w)
        clean_tokens_list += clean_tokens
        clean_text_list.append(' '.join(clean_tokens)) 
        
    # 建立詞彙表物件
    counter = Counter(clean_tokens_list)    
    sorted_by_freq_tuples = sorted(counter.items(), 
                                   key=lambda x: x[1], reverse=True)
    ordered_dict = OrderedDict(sorted_by_freq_tuples)
    vocab_object = torchtext.vocab.vocab(ordered_dict, specials=["<unk>"])
    vocab_object.set_default_index(vocab_object["<unk>"])
    
    # 將輸入字串轉為索引值：自詞彙表物件查詢索引值
    clean_index_list = []
    for clean_tokens_list in clean_text_list:
        clean_index_list.append(
            vocab_object.lookup_indices(clean_tokens_list.split(' ')))
    
    # 輸出 詞彙表物件、去除標點符號的字串陣列、字串陣列的索引值
    return vocab_object, clean_text_list, clean_index_list

## 測試

In [358]:
docs = ['Well done!',
        'Good work',
        'Great effort',
        'nice work',
        'Excellent!',
        'Weak',
        'Poor effort!',
        'not good',
        'poor work',
        'Could have done better.']

vocab_object, clean_text_list, clean_index_list = create_vocabulary(docs)
vocab_object.get_itos()

['<unk>',
 'work',
 'done',
 'good',
 'effort',
 'poor',
 'well',
 'great',
 'nice',
 'excellent',
 'weak',
 'not',
 'could',
 'have',
 'better']

In [359]:
clean_text_list 

['well done',
 'good work',
 'great effort',
 'nice work',
 'excellent',
 'weak',
 'poor effort',
 'not good',
 'poor work',
 'could have done better']

In [360]:
clean_index_list

[[6, 2],
 [3, 1],
 [7, 4],
 [8, 1],
 [9],
 [10],
 [5, 4],
 [11, 3],
 [5, 1],
 [12, 13, 2, 14]]

# 整合以上功能，實作一個簡單的案例，說明相關的處理程序

## 建立詞彙表：整理輸入語句，截長補短，使語句長度一致。

In [361]:
maxlen = 4      # 語句最大字數
# 測試資料
docs = ['Well done!',
        'Good work',
        'Great effort',
        'nice work',
        'Excellent!',
        'Weak',
        'Poor effort!',
        'not good',
        'poor work',
        'Could have done better']

vocab_object, clean_text_list, clean_index_list = create_vocabulary(docs)

# 若字串過長，刪除多餘單字
clean_index_list = torchtext.functional.truncate(clean_index_list, maxlen)

# 若字串長度不足，後面補 0
while len(clean_index_list[0]) < maxlen:
    clean_index_list[0] += [0]
torchtext.functional.to_tensor(clean_index_list, 0) # 0:不足補0

tensor([[ 6,  2,  0,  0],
        [ 3,  1,  0,  0],
        [ 7,  4,  0,  0],
        [ 8,  1,  0,  0],
        [ 9,  0,  0,  0],
        [10,  0,  0,  0],
        [ 5,  4,  0,  0],
        [11,  3,  0,  0],
        [ 5,  1,  0,  0],
        [12, 13,  2, 14]])

In [362]:
# 測試 
embeds = nn.Embedding(vocab_object.__len__(), 5) 
X = torchtext.functional.to_tensor(clean_index_list, 0) # 0:不足補0
embed_output = embeds(X)
print(embed_output.shape)

torch.Size([10, 4, 5])


## 加上完全連接層(Linear)

In [366]:
class RecurrentNet(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.fc = nn.Linear(embed_dim * maxlen, num_class) # 要乘以 maxlen
        self.embed_dim = embed_dim
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text):
        embedded = self.embedding(text)
        out = embedded.reshape(embedded.size(0), -1) # 轉換成1維
        return self.fc(out)

model = RecurrentNet(vocab_object.__len__(), 10, 1)

## 另一種寫法，使用EmbeddingBag

In [363]:
class RecurrentNet(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = nn.Linear(embed_dim, num_class)
        self.embed_dim = embed_dim
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text):
        embedded = self.embedding(text)
        return self.fc(embedded)

model = RecurrentNet(vocab_object.__len__(), 10, 1)

In [367]:
# 定義 10 個語句的正面(1)或負面(0)的情緒
y = torch.FloatTensor([1,1,1,1,1,0,0,0,0,0])
X = torchtext.functional.to_tensor(clean_index_list, 0) # 0:不足補0

# 指定優化器、損失函數
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

# 模型訓練
for epoch in range(1000):
    outputs = model.forward(X) #forward pass
    optimizer.zero_grad() 
    loss = criterion(outputs.reshape(-1), y)
    loss.backward() 
    optimizer.step() 
    if epoch % 100 == 0:
        #print(outputs.shape)
        print(f"Epoch: {epoch}, loss: {loss.item():1.5f}")

Epoch: 0, loss: 0.72728
Epoch: 100, loss: 0.06390
Epoch: 200, loss: 0.00598
Epoch: 300, loss: 0.00140
Epoch: 400, loss: 0.00052
Epoch: 500, loss: 0.00016
Epoch: 600, loss: 0.00004
Epoch: 700, loss: 0.00001
Epoch: 800, loss: 0.00000
Epoch: 900, loss: 0.00000


In [368]:
# 模型評估
model.eval()
model(X)

tensor([[ 1.0000e+00],
        [ 9.9983e-01],
        [ 1.0002e+00],
        [ 9.9990e-01],
        [ 1.0000e+00],
        [-3.0272e-05],
        [-3.2104e-04],
        [ 1.9193e-05],
        [ 3.5113e-04],
        [-7.1526e-07]], grad_fn=<AddmmBackward0>)

In [369]:
# 測試資料
test_docs = ['great effort', 'well done',
        'poor effort']

# 轉成數值 
clean_index_list = []
for text in test_docs:
    clean_index_list.append(vocab_object.lookup_indices(text.split(' ')))
while len(clean_index_list[0]) < maxlen:
    clean_index_list[0] += [0]

clean_index_list = torchtext.functional.truncate(clean_index_list, maxlen)    
X = torchtext.functional.to_tensor(clean_index_list, 0) # 0:不足補0
model(X)

tensor([[ 1.0002e+00],
        [ 1.0000e+00],
        [-3.2104e-04]], grad_fn=<AddmmBackward0>)

## 使用詞向量(Word2Vec)

## 讀取 GloVe 50維的詞向量，轉換為GloVe 50維的詞向量

In [302]:
# https://pytorch.org/text/stable/vocab.html#glove
examples = ['great']
vec = torchtext.vocab.GloVe(name='6B', dim=50)
ret = vec.get_vecs_by_tokens(examples, lower_case_backup=True)
ret

tensor([[-0.0266,  1.3357, -1.0280, -0.3729,  0.5201, -0.1270, -0.3543,  0.3782,
         -0.2972,  0.0939, -0.0341,  0.9296, -0.1402, -0.6330,  0.0208, -0.2153,
          0.9692,  0.4765, -1.0039, -0.2401, -0.3632, -0.0048, -0.5148, -0.4626,
          1.2447, -1.8316, -1.5581, -0.3747,  0.5336,  0.2088,  3.2209,  0.6455,
          0.3744, -0.1766, -0.0242,  0.3379, -0.4190,  0.4008, -0.1145,  0.0512,
         -0.1521,  0.2986, -0.4405,  0.1109, -0.2463,  0.6625, -0.2695, -0.4966,
         -0.4162, -0.2549]])

In [303]:
vec.vectors.size()

torch.Size([400000, 50])

In [304]:
vec.stoi['great']

353

## Embedding 不需訓練，直接設定嵌入層權重

In [338]:
class RecurrentNet(nn.Module):
    def __init__(self, weights_matrix, num_embeddings, embedding_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(num_embeddings, embedding_dim)
        # 設定嵌入層權重
        self.embedding.load_state_dict({'weight': weights_matrix})
        self.fc = nn.Linear(embedding_dim, num_class)

    def forward(self, text):
        embedded = self.embedding(text)
        return self.fc(embedded)

## 測試資料轉換

In [339]:
docs = ['Well done!',
        'Good work',
        'Great effort',
        'nice work',
        'Excellent!',
        'Weak',
        'Poor effort!',
        'not good',
        'poor work',
        'Could have done better']

# 將詞彙表轉為詞向量
clean_text_list = []
clean_tokens_list = []
for i, text in enumerate(docs):
    tokens = tokenizer(text.lower()) 
    clean_tokens = []
    for w in tokens:
        if w not in stopwords:
            clean_tokens.append(w)
    clean_tokens_list += clean_tokens   
    clean_text_list.append(clean_tokens)  
    tokens_vec = vec.get_vecs_by_tokens(clean_tokens)
vocab_list = list(set(clean_tokens_list))            
weights_matrix = vec.get_vecs_by_tokens(vocab_list)

In [340]:
# 定義 10 個語句的正面(1)或負面(0)的情緒
y = torch.FloatTensor([1,1,1,1,1,0,0,0,0,0])
X = torch.LongTensor(np.zeros((len(docs), maxlen)))
for i, item in enumerate(clean_text_list):
    for j, token in enumerate(item):
        if token in vocab_list:
            X[i, j] = vocab_list.index(token)
X

tensor([[ 9,  6,  0,  0],
        [10,  4,  0,  0],
        [13, 12,  0,  0],
        [ 0,  4,  0,  0],
        [ 7,  0,  0,  0],
        [ 8,  0,  0,  0],
        [ 2, 12,  0,  0],
        [ 5, 10,  0,  0],
        [ 2,  4,  0,  0],
        [ 3, 11,  6,  1]])

In [341]:
vocab_list

['nice',
 'better',
 'poor',
 'could',
 'work',
 'not',
 'done',
 'excellent',
 'weak',
 'well',
 'good',
 'have',
 'effort',
 'great']

In [342]:
# 建立模型物件
model = RecurrentNet(torch.FloatTensor(weights_matrix), len(vocab_list), 50, 1)

# 指定優化器、損失函數
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

# 模型訓練
for epoch in range(1000):
    outputs = model.forward(X) #forward pass
    optimizer.zero_grad() 
    loss = criterion(outputs.reshape(-1), y)
    loss.backward() 
    optimizer.step() 
    if epoch % 100 == 0:
        #print(outputs.shape)
        print(f"Epoch: {epoch}, loss: {loss.item():1.5f}")

Epoch: 0, loss: 1.03547
Epoch: 100, loss: 0.09790
Epoch: 200, loss: 0.02937
Epoch: 300, loss: 0.00643
Epoch: 400, loss: 0.00280
Epoch: 500, loss: 0.00141
Epoch: 600, loss: 0.00067
Epoch: 700, loss: 0.00031
Epoch: 800, loss: 0.00013
Epoch: 900, loss: 0.00006


In [343]:
# 模型評估
model.eval()
model(X)

tensor([[ 1.0005e+00],
        [ 1.0003e+00],
        [ 1.0043e+00],
        [ 9.9065e-01],
        [ 1.0017e+00],
        [ 1.3793e-03],
        [-6.5045e-03],
        [-4.8908e-05],
        [ 9.2722e-03],
        [-1.0637e-04]], grad_fn=<AddmmBackward0>)

In [344]:
# 測試資料
test_docs = ['great effort', 'well done',
        'poor effort']

# 轉成數值 
X = torch.LongTensor(np.zeros((len(test_docs), maxlen)))
clean_text_list = []
for i, text in enumerate(test_docs):
    tokens = tokenizer(text.lower()) 
    clean_tokens = []
    for w in tokens:
        if w not in stopwords:
            clean_tokens.append(w)
    clean_text_list.append(clean_tokens)  

for i, item in enumerate(clean_text_list):
    for j, token in enumerate(item):
        if token in vocab_list:
            X[i, j] = vocab_list.index(token)

# 預測            
model.eval()        
model(X)

tensor([[ 1.0043],
        [ 1.0005],
        [-0.0065]], grad_fn=<AddmmBackward0>)

## 將整個詞向量設定為嵌入層權重

In [295]:
class RecurrentNet2(nn.Module):
    def __init__(self, vec, embedding_dim, num_class):
        super().__init__()
        # 將整個詞向量設定為嵌入層權重，且嵌入層設為不訓練
        self.embedding = nn.EmbeddingBag.from_pretrained(vec, freeze=True)
        self.fc = nn.Linear(embedding_dim, num_class)

    def forward(self, text):
        embedded = self.embedding(text)
        return self.fc(embedded)
    
model = RecurrentNet2(vec.vectors, vec.dim, 1)

In [296]:
# 測試資料
docs = ['Well done!',
        'Good work',
        'Great effort',
        'nice work',
        'Excellent!',
        'Weak',
        'Poor effort!',
        'not good',
        'poor work',
        'Could have done better']

# 轉成數值 
X = torch.LongTensor(np.zeros((len(docs), maxlen)))

for i, text in enumerate(docs):
    tokens = tokenizer(text.lower()) 
    clean_tokens = []
    j=0
    for w in tokens:
        if w not in stopwords:
            # 轉成詞向量索引值 
            X[i, j] = vec.stoi[w]
            j+=1
X

tensor([[ 143,  751,    0,    0],
        [ 219,  161,    0,    0],
        [ 353,  968,    0,    0],
        [3082,  161,    0,    0],
        [4345,    0,    0,    0],
        [2690,    0,    0,    0],
        [ 992,  968,    0,    0],
        [  36,  219,    0,    0],
        [ 992,  161,    0,    0],
        [  94,   33,  751,  439]])

In [297]:
# 指定優化器、損失函數
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

# 模型訓練
for epoch in range(1000):
    outputs = model.forward(X) #forward pass
    optimizer.zero_grad() 
    loss = criterion(outputs.reshape(-1), y)
    loss.backward() 
    optimizer.step() 
    if epoch % 100 == 0:
        #print(outputs.shape)
        print(f"Epoch: {epoch}, loss: {loss.item():1.5f}")

model.eval()        
model(X)

Epoch: 0, loss: 1.26617
Epoch: 100, loss: 0.12657
Epoch: 200, loss: 0.07799
Epoch: 300, loss: 0.05655
Epoch: 400, loss: 0.04253
Epoch: 500, loss: 0.03269
Epoch: 600, loss: 0.02547
Epoch: 700, loss: 0.01998
Epoch: 800, loss: 0.01569
Epoch: 900, loss: 0.01227


tensor([[ 0.8730],
        [ 0.9340],
        [ 1.0150],
        [ 0.9696],
        [ 0.9747],
        [-0.0099],
        [-0.0561],
        [ 0.2156],
        [ 0.1366],
        [-0.0677]], grad_fn=<AddmmBackward0>)

In [299]:
# 測試資料
test_docs = ['great job', 'well done',
        'poor job']

# 轉成數值 
X = torch.LongTensor(np.zeros((len(test_docs), maxlen)))
for i, text in enumerate(test_docs):
    tokens = tokenizer(text.lower()) 
    clean_tokens = []
    j=0
    for w in tokens:
        if w not in stopwords:
            X[i, j] = vec.stoi[w]
            j+=1
X

tensor([[353, 664,   0,   0],
        [143, 751,   0,   0],
        [992, 664,   0,   0]])

In [301]:
# 預測            
model.eval()        
model(X)

tensor([[ 0.6623],
        [ 0.8730],
        [-0.4088]], grad_fn=<AddmmBackward0>)